In [2]:
import pandas as pd
import pickle
import os 

pd.set_option('display.max_rows',10000)
pd.set_option('display.min_rows',2000)
pd.set_option('display.column_space',30)
pd.set_option('display.max_colwidth',150)
pd.set_option('display.expand_frame_repr',True)

_end_dates    = pd.Series(pd.date_range(start="2020-10-08", end="2021-03-31", freq="D", tz='America/New_York'))
_start_dates  = _end_dates - pd.Timedelta(days=7)
_center_dates = _end_dates - pd.Timedelta(days=3.5)
end_dates     = _end_dates.apply(lambda x: x.timestamp())
center_dates  = _center_dates.apply(lambda x: x.timestamp())
start_dates   = _start_dates.apply(lambda x: x.timestamp())
_end_dates    = _end_dates.apply(lambda x:str(x).split(' ')[0])
_center_dates = _center_dates.apply(lambda x:str(x).split(' ')[0])
_start_dates  = _start_dates.apply(lambda x:str(x).split(' ')[0])

id_l = '/home/casonk/path/mmani_root/mmani0/shared_data/hot/push_file/IDL/'

j=1
k=21
r=2

In [3]:
date = _center_dates[25]
with open((id_l + date + ('/LC_{}_{}_{}.pkl').format(j, k, r)), 'rb') as lch:
    lc = pickle.load(lch)

In [5]:
pd.read_pickle((id_l + date + ('/MAXIMALIST_DF_{}_{}_{}.pkl').format(j, k, r)))
# pd.read_pickle((id_l + date + ('/TRIMMED_DF_{}_{}.pkl').format(j, k)))

,Source,Target,Subreddit,Count,Source_Level,Target_Level
9,jfqt1,mrcs40-bot,csci040temp,9429,7898,7898
53,sophie-cs40,easeany,csci040temp,2651,7898,7898
60,shitpisscum1312,thejoshster,perfectlycutscreens,2510,206,206
61,cooldownbot,thejoshster,perfectlycutscreens,2507,206,206
62,fuckcooldownbot2,thejoshster,perfectlycutscreens,2501,206,206
80,fuckthisshitbot41,thejoshster,perfectlycutscreens,2122,206,206
145,cs40bot,li_bot_cs40,csci040temp,1470,7898,7898
148,lab-bot,li_bot_cs40,csci040temp,1429,7898,7898
169,namesbotjamesbot,li_bot_cs40,csci040temp,1307,7898,7898
187,wilfveelveel,winterj55,askreddit,1229,76,76
